In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



In [6]:
arabic_dialect_dataset = pd.read_csv('./arabic_dialect.csv')
arabic_dialect_dataset

,id,user,sentence,tokenized_sentence,dialect,dialect_AE,dialect_BH,dialect_DZ,dialect_EG,dialect_IQ,...,dialect_LY,dialect_MA,dialect_OM,dialect_PL,dialect_QA,dialect_SA,dialect_SD,dialect_SY,dialect_TN,dialect_YE
0,1175358310087892992,@Nw8ieJUwaCAAreT,لكن بالنهاية .. ينتفض .. يغير .,"['لكن', 'بالنهاية', '..', 'ينتفض', '..', 'يغير...",IQ,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1175416117793349632,@7zNqXP0yrODdRjK,يعني هذا محسوب على البشر .. حيونه ووحشيه .. وت...,"['يعني', 'هذا', 'محسوب', 'على', 'البشر', '..',...",IQ,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1175450108898565888,@KanaanRema,مبين من كلامه خليجي,"['مبين', 'من', 'كلامه', 'خليجي']",IQ,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1175471073770573824,@HAIDER76128900,يسلملي مرورك وروحك الحلوه💐,"['يسلملي', 'مرورك', 'وروحك', 'الحلوه💐']",IQ,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1175496913145217024,@hmo2406,وين هل الغيبه اخ محمد 🌸🌺,"['وين', 'هل', 'الغيبه', 'اخ', 'محمد', '🌸🌺']",IQ,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458196,1022409931029458944,@jolnar121,السحله ضيفي ي بتطلع لك سحليه😅😅,"['السحله', 'ضيفي', 'ي', 'بتطلع', 'لك', 'سحليه😅😅']",BH,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
458197,1057418989293485952,@mycousinvinnyys,@hanyamikhail1 متهيالي دي شكولاته الهالوين فين...,"['@', 'hanyamikhail1', 'متهيالي', 'دي', 'شكولا...",EG,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
458198,1055620304465215616,@MahmoudWaked7,@maganenoo في طريق مطروح مركز بهيج والمركز الي...,"['@', 'maganenoo', 'في', 'طريق', 'مطروح', 'مرك...",EG,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
458199,1057418989293485952,@mycousinvinnyys,@hanyamikhail1 متهيالي دي شكولاته الهالوين فين...,"['@', 'hanyamikhail1', 'متهيالي', 'دي', 'شكولا...",EG,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
